In [1]:
source("/data/srlab/ik936/Fiona/utils.R")
source("/data/srlab/ik936/Fiona/libs.R")


# Load data

In [4]:
library(presto)
library(Matrix)
sumOverRowNames <- function(X) {
    name_factors <- factor(row.names(X))
    res <- presto:::sumGroups.dgCMatrix(X, name_factors)
    row.names(res) <- levels(name_factors)[1:nrow(res)]
    colnames(res) <- colnames(X)
    return(res)
}

read10x <- function(run, suffix) {
    barcode.loc <- list.files(run, pattern = 'barcodes', full.names = TRUE)
    gene.loc <- list.files(run, pattern = 'features.tsv|genes.tsv', full.names = TRUE)
    matrix.loc <- list.files(run, pattern = 'matrix.mtx', full.names = TRUE)

    data <- readMM(file = matrix.loc) %>% as("dgCMatrix")
    cell.names <- readLines(barcode.loc)
    cell.names <- gsub("-1$", "", cell.names)
    if (!missing(suffix)) {
        cell.names %<>% paste(suffix, sep = "_")
    }
        
    gene.names <- fread(gene.loc, header = FALSE)$V1
    row.names(data) <- gene.names
    colnames(data) <- cell.names
  
    return(as(data, "dgCMatrix"))
#     return(as(sumOverRowNames(data), "dgCMatrix"))
}


## metadata

In [5]:
meta_data <- fread('data/broad/all.meta.txt')
meta_data <- meta_data[2:nrow(meta_data), ] %>% data.frame()
meta_data <- meta_data %>%
    dplyr::mutate(Cluster = gsub('\\+', 'p', Cluster)) %>% 
    dplyr::mutate(Cluster = gsub(' ', '', Cluster)) %>% 
    dplyr::mutate(Cluster = gsub('-', 'Health', Cluster))

row.names(meta_data) <- meta_data$NAME

In [6]:
head(meta_data)

,NAME,Cluster,nGene,nUMI,Subject,Health,Location
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
N7.EpiA.AAACATACACACTG,N7.EpiA.AAACATACACACTG,TA1,328,891,N7,Non-inflamed,Epi
N7.EpiA.AAACCGTGCATCAG,N7.EpiA.AAACCGTGCATCAG,TA1,257,663,N7,Non-inflamed,Epi
N7.EpiA.AAACGCACAATCGC,N7.EpiA.AAACGCACAATCGC,TA2,300,639,N7,Non-inflamed,Epi
N7.EpiA.AAAGATCTAACCGT,N7.EpiA.AAAGATCTAACCGT,EnterocyteProgenitors,250,649,N7,Non-inflamed,Epi
N7.EpiA.AAAGATCTAGGCGA,N7.EpiA.AAAGATCTAGGCGA,EnterocyteProgenitors,284,769,N7,Non-inflamed,Epi
N7.EpiA.AAAGCCTGCTCGAA,N7.EpiA.AAAGCCTGCTCGAA,EnterocyteProgenitors,339,951,N7,Non-inflamed,Epi


## exprs

In [7]:
library(singlecellmethods)
exprs_fib <- read10x('/data/srlab/ik936/Fiona/data/broad/fib') %>% 
    normalizeData(scaling_factor = 1e4, method = 'log')
exprs_imm <- read10x('/data/srlab/ik936/Fiona/data/broad/imm') %>% 
    normalizeData(scaling_factor = 1e4, method = 'log')
exprs_epi <- read10x('/data/srlab/ik936/Fiona/data/broad/epi') %>% 
    normalizeData(scaling_factor = 1e4, method = 'log')

In [8]:
## adds rows that are missing in each matrix
## fills in missing rows with 0s
cbind_incomplete <- function(X, Y) {
    g1 <- row.names(X)
    g2 <- row.names(Y)
    genes_all <- union(g1, g2)
    
    genes_add_to_Y <- setdiff(g1, g2)
    Y_add <- rsparsematrix(length(genes_add_to_Y), ncol(Y), 0)
    row.names(Y_add) <- genes_add_to_Y
    Y <- Matrix::rbind2(Y, Y_add)[genes_all, ]
    
    genes_add_to_X <- setdiff(g2, g1)
    X_add <- rsparsematrix(length(genes_add_to_X), ncol(X), 0)
    row.names(X_add) <- genes_add_to_X
    X <- Matrix::rbind2(X, X_add)[genes_all, ]    
    
    Matrix::cbind2(X, Y)
}


In [9]:
exprs_all <- Reduce(cbind_incomplete, list(exprs_fib, exprs_imm, exprs_epi))

In [10]:
dim(exprs_all)

[1]  21784 365492

In [11]:
saveRDS(exprs_all, '/data/srlab/ik936/Fiona/data/broad/exprs_norm_all.rds')

# Load module data

In [7]:
exprs_all <- readRDS('/data/srlab/ik936/Fiona/data/broad/exprs_norm_all.rds')

In [8]:
modules <- fread('data/WGCNA_all.csv')[, N := .N, by = moduleColor][N < 2000]

In [10]:
mod_df <- data.table(geneName = row.names(exprs_all)) %>% 
    dplyr::left_join(modules) 
genes_use <- which(!is.na(mod_df$moduleColor))

Joining, by = "geneName"



In [11]:
length(genes_use)

[1] 6324

## joint expression matrix

    normalize
    scale

In [12]:
dim(exprs_all)

[1]  21784 365492

In [17]:
exprs_scaled <- exprs_all[genes_use, ] %>% 
    scaleData() 

In [17]:
exprs_scaled %>% saveRDS('/data/srlab/ik936/Fiona/data/broad/exprs_scaled_all.rds')

In [ ]:
dim(exprs_scaled)

## Sum over types and modules

In [ ]:
exprs_scaled <- readRDS('/data/srlab/ik936/Fiona/data/broad/exprs_scaled_all.rds')

In [15]:
head(meta_data)

,NAME,Cluster,nGene,nUMI,Subject,Health,Location
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
N7.EpiA.AAACATACACACTG,N7.EpiA.AAACATACACACTG,TA1,328,891,N7,Non-inflamed,Epi
N7.EpiA.AAACCGTGCATCAG,N7.EpiA.AAACCGTGCATCAG,TA1,257,663,N7,Non-inflamed,Epi
N7.EpiA.AAACGCACAATCGC,N7.EpiA.AAACGCACAATCGC,TA2,300,639,N7,Non-inflamed,Epi
N7.EpiA.AAAGATCTAACCGT,N7.EpiA.AAAGATCTAACCGT,EnterocyteProgenitors,250,649,N7,Non-inflamed,Epi
N7.EpiA.AAAGATCTAGGCGA,N7.EpiA.AAAGATCTAGGCGA,EnterocyteProgenitors,284,769,N7,Non-inflamed,Epi
N7.EpiA.AAAGCCTGCTCGAA,N7.EpiA.AAAGCCTGCTCGAA,EnterocyteProgenitors,339,951,N7,Non-inflamed,Epi


In [16]:
ncol(exprs_scaled)
nrow(meta_data)

[1] 365492

[1] 365492

In [28]:
head(meta_data)

,NAME,Cluster,nGene,nUMI,Subject,Health,Location
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
N7.EpiA.AAACATACACACTG,N7.EpiA.AAACATACACACTG,TA1,328,891,N7,Non-inflamed,Epi
N7.EpiA.AAACCGTGCATCAG,N7.EpiA.AAACCGTGCATCAG,TA1,257,663,N7,Non-inflamed,Epi
N7.EpiA.AAACGCACAATCGC,N7.EpiA.AAACGCACAATCGC,TA2,300,639,N7,Non-inflamed,Epi
N7.EpiA.AAAGATCTAACCGT,N7.EpiA.AAAGATCTAACCGT,EnterocyteProgenitors,250,649,N7,Non-inflamed,Epi
N7.EpiA.AAAGATCTAGGCGA,N7.EpiA.AAAGATCTAGGCGA,EnterocyteProgenitors,284,769,N7,Non-inflamed,Epi
N7.EpiA.AAAGCCTGCTCGAA,N7.EpiA.AAAGCCTGCTCGAA,EnterocyteProgenitors,339,951,N7,Non-inflamed,Epi


In [44]:
y <- meta_data[colnames(exprs_scaled), ]$Cluster
idx <- which(!is.na(y) & y != 'NA')
y <- factor(y[idx])
# exprs_use <- exprs_norm[, idx]

mod_df <- data.table(geneName = row.names(exprs_scaled)) %>% 
    dplyr::left_join(modules) 
genes_use <- which(!is.na(mod_df$moduleColor))

mod_df <- data.frame(mod_df)
rownames(mod_df) <- mod_df$geneName


Joining, by = "geneName"



In [45]:
# exprs_fib_mean <- presto:::sumGroups(exprs_fib, y, 1)
exprs_fib_mean <- presto:::sumGroups(exprs_scaled, y, 1)
exprs_fib_mean <- crossprod(exprs_fib_mean, diag(1 / table(y)))
row.names(exprs_fib_mean) <- genes_use #row.names(obj$exprs_norm)
colnames(exprs_fib_mean) <- levels(y)


Get cluster x module

In [47]:
y <- factor(mod_df[genes_use, 'moduleColor'])

mod_means <- presto::sumGroups(exprs_fib_mean, y)
mod_means <- crossprod(mod_means, diag(1 / table(y))) %>% t
row.names(mod_means) <- levels(y)
colnames(mod_means) <- colnames(exprs_fib_mean)


In [48]:
dim(mod_means)

[1] 38 51

In [46]:
mod_means %>% saveRDS('/data/srlab/ik936/Fiona/data/scores_broad.rds')